#  Construction of hmm profile using a set of sequences 

## states_define function outputs all hidden states in the profile hmm graph given length(sequence)-1 as input  , by sequence I mean a alignment ex. "AT_GC_"

In [1]:
def states_define(nn):
    hidden_states = ["S0", 'I0']
    for i in range(nn):
        hidden_states.append('M'+str(i+1))
        hidden_states.append('D'+str(i+1))
        hidden_states.append('I'+str(i+1))
    hidden_states.append('E')
    return hidden_states

## Returns the index of a element from the array, the element is present in

In [2]:
def where(a,b):
    x=list(idx for idx, val in np.ndenumerate(a) if val==b)[0][0]
    return(x)

## counts the no. of gaps in each column from a array containing all the alignments , if the no. of gaps is greater than a particular threshold value it sets the array x at the same index as column to 0 and if no. of gaps is less than a threshold it sets it as 1.

In [3]:
import numpy as np
def check(alignments,threshold):
    ll=len(alignments[0])
    x=np.ones(ll)
    l=len(alignments)
    for i in range(ll):
        s=0
        for j in range(l):
            if alignments[j][i]=="-":
                s+=1/l
        if s>threshold:
            x[i]=0.0
    return(x)

# construction of profile hmm, using alignments provided . every time a edge is traversed 1 is added to the edge of a profile hmm graph.Please refer to the image attached with the mail, where the process is pictorically explained. basically each alignment is a path in the graph 

In [4]:
import numpy as np
def profile_hmm(observations,alignments,hidden_states,true_matrix):
    trans_mat=np.zeros((len(hidden_states),len(hidden_states)))
    emiss_mat=np.zeros((len(hidden_states),len(observations)))
    length=len(alignments[0])
    for seq in alignments:
        cur_hidden_state="S0"
        for i in range(length):
            if true_matrix[i]:
                if seq[i] in observations:
                    new_hidden_state="M"+str(int(cur_hidden_state[1:])+1)
                    trans_mat[where(hidden_states,cur_hidden_state)][where(hidden_states,new_hidden_state)]+=1
                    emiss_mat[where(hidden_states,new_hidden_state)][where(observations,seq[i])]+=1
                    cur_hidden_state=new_hidden_state
                else:
                    new_hidden_state = 'D'+str(int(cur_hidden_state[1:])+1)
                    trans_mat[where(hidden_states,cur_hidden_state)][where(hidden_states,new_hidden_state)] += 1
                    cur_hidden_state = new_hidden_state     
            else:
                if seq[i] in observations:
                    new_hidden_state = 'I'+str(int(cur_hidden_state[1:]))
                    trans_mat[where(hidden_states,cur_hidden_state)][where(hidden_states,new_hidden_state)] += 1
                    emiss_mat[where(hidden_states,new_hidden_state)][where(observations,seq[i])] += 1
                    cur_hidden_state = new_hidden_state
        new_hidden_state = 'E'
        trans_mat[where(hidden_states,cur_hidden_state)][where(hidden_states,new_hidden_state)] += 1
    return trans_mat, emiss_mat   

# adds puedocount and normailzation 

In [5]:
def add_pseudo(hid, inc, trans, emit, pseud):
    for index, item in enumerate(hid):
        if item == 'E':
            continue
        if item[0] in ['I', 'M']:
            new = [x+pseud for x in emit[index]]
            emit[index] = new
        s = item[1:]
        if int(s) < inc:
            possible_moves = ['I'+s, 'M'+str(int(s)+1), 'D'+str(int(s)+1)]
            for new_item in possible_moves:
                trans[index][where(hid,new_item)] += pseud
        elif int(s) == inc:
            possible_moves = ['I'+s, 'E']
            for new_item in possible_moves:
                trans[index][where(hid,new_item)] += pseud
    for i, line in enumerate(trans):
        s = sum(line)
        if s > 0:
            normalized = [x/s for x in line]
            trans[i] = normalized

    for i, line in enumerate(emit):
        s = sum(line)
        if s > 0:
            normalized = [x/s for x in line]
            emit[i] = normalized
    return trans, emit

In [79]:
def add_pseudo(hid, inc, trans, emit, pseud):
    for i, line in enumerate(trans):
        s = sum(line)
        if s > 0:
            normalized = [x/s for x in line]
            trans[i] = normalized

    for i, line in enumerate(emit):
        s = sum(line)
        if s > 0:
            normalized = [x/s for x in line]
            emit[i] = normalized

    for index, item in enumerate(hid):
        if item == 'E':
            continue
        if item[0] in ['I', 'M']:
            new = [x+pseud for x in emit[index]]
            emit[index] = new
        s = item[1:]
        if int(s) < inc:
            possible_moves = ['I'+s, 'M'+str(int(s)+1), 'D'+str(int(s)+1)]
            for new_item in possible_moves:
                trans[index][hid.index(new_item)] += pseud
        elif int(s) == inc:
            possible_moves = ['I'+s, 'E']
            for new_item in possible_moves:
                trans[index][hid.index(new_item)] += pseud
    return trans, emit

# viterbi algorithm for profile hmm

In [6]:
from math import log10
from operator import itemgetter
from copy import copy
def generate_viterbi(sequence, inc, hid, tran, emit,states):
    viterbi = list()
    column = dict()
    column['S0'] = (0, 0, 'Source')
    column['D1'] = (log10(tran[hid.index('S0')][hid.index('D1')]), 0, 'S0')
    for i in range(1, inc+1):
        column['D'+str(i+1)] = (log10(tran[hid.index('D'+str(i))][hid.index('D'+str(i+1))]) + column['D'+str(i)][0], 0, 'D'+str(i))
    viterbi.append(copy(column))
    for j, lett in enumerate(sequence):
        column = dict()
        if not j:
            column['I0'] = (log10(tran[hid.index('S0')][hid.index('I0')]) + log10(emit[hid.index('I0')][states.index(lett)]), 0, 'S0')
            column['M1'] = (log10(tran[hid.index('S0')][hid.index('M1')]) + log10(emit[hid.index('M1')][states.index(lett)]), 0, 'S0')
            column['D1'] = (column['I0'][0] + log10(tran[hid.index('I0')][hid.index('D1')]), 1, 'I0')
            for stat in hid[4:-1]:
                l = stat[0]
                ind = int(stat[1:])
                if l == 'I':
                    fath = 'D'+str(ind)
                    column[stat] = (viterbi[j][fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]) + log10(emit[hid.index(stat)][states.index(lett)]), 0, fath)
                if l == 'M':
                    fath = 'D'+str(ind-1)
                    column[stat] = (viterbi[j][fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]) + log10(emit[hid.index(stat)][states.index(lett)]), 0, fath)
                if l == 'D':
                    temp = str(ind-1)
                    fathers = [tt+temp for tt in ['I', 'M', 'D']]
                    proba = [(column[fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]), j+1, fath) for fath in fathers]
                    column[stat] = max(proba, key=itemgetter(0))
            viterbi.append(copy(column))
        else:
            column['I0'] = (viterbi[j]['I0'][0] + log10(tran[hid.index('I0')][hid.index('I0')]) + log10(emit[hid.index('I0')][states.index(lett)]), j, 'I0')
            column['M1'] = (viterbi[j]['I0'][0] + log10(tran[hid.index('I0')][hid.index('M1')]) + log10(emit[hid.index('M1')][states.index(lett)]), j, 'I0')
            column['D1'] = (column['I0'][0] + log10(tran[hid.index('I0')][hid.index('D1')]), 1, 'I0')
            for stat in hid[4:-1]:
                l = stat[0]
                ind = int(stat[1:])
                if l == 'I':
                    temp = str(ind)
                    fathers = [tt+temp for tt in ['I', 'M', 'D']]
                    proba = [(viterbi[j][fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]) + log10(emit[hid.index(stat)][states.index(lett)]), j, fath) for fath in fathers]
                if l == 'M':
                    temp = str(ind-1)
                    fathers = [tt+temp for tt in ['I', 'M', 'D']]
                    proba = [(viterbi[j][fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]) + log10(emit[hid.index(stat)][states.index(lett)]), j, fath) for fath in fathers]
                if l == 'D':
                    temp = str(ind-1)
                    fathers = [tt+temp for tt in ['I', 'M', 'D']]
                    proba = [(column[fath][0] + log10(tran[hid.index(fath)][hid.index(stat)]), j+1, fath) for fath in fathers]
                column[stat] = max(proba, key=itemgetter(0))
            viterbi.append(copy(column))
    return viterbi

In [45]:
ali=np.array(["ATGC","ATGC","AT-C","ATGG","AT--","-TGC","AAGC","ATTC","ATCC","ATGC","ATGC"])

In [80]:
xx=[]
for i in ali:
    x=[]
    for j in range(len(ali[0])):
        x.append(i[j])
    xx.append(x)

In [48]:
t=0.0 # the threshold 

obs=["A","T","G","C"] # observable states

true_mat=check(xx,t) # truth matrix based on no. og gaps in each column of alignments array

sta=states_define(len(xx[0])) # finds all hidden states in hmm profile array

x,y=profile_hmm(obs,xx,sta,true_mat)

x1,y1=add_pseudo(sta,len(xx[0]), x, y, 0.5)

string=["A","G","C","C"]

v=generate_viterbi(string,len(xx[0])-1, sta, x1, y1,obs)

path = list()
final_states = [tt+str(len(xx[0])) for tt in ['M', 'D', 'I']]
proba = [(v[-1][fath][0] + log10(x1[sta.index(fath)][sta.index('E')]), len(string), fath) for fath in final_states]
exit_state = max(proba, key=itemgetter(0))
pointer = exit_state[1], exit_state[2]
while True:
    path.append(pointer[1])
    back = v[pointer[0]][pointer[1]]
    pointer = back[1], back[2]
    if pointer[1] == 'S0':
        break

print(' '.join(path[::-1]))

I0 D1 M2 M3 M4


In [51]:
t=0.205 # the threshold 

obs=["A","T","G","C"] # observable states

true_mat=check(xx,t) # truth matrix based on no. og gaps in each column of alignments array

sta=states_define(len(xx[0])) # finds all hidden states in hmm profile array

x,y=profile_hmm(obs,xx,sta,true_mat)

x1,y1=add_pseudo(sta,len(xx[0]), x, y, 0.5)

string=["A","G","C"] # one deletion

v=generate_viterbi(string,len(xx[0])-1, sta, x1, y1,obs)

path = list()
final_states = [tt+str(len(xx[0])) for tt in ['M', 'D', 'I']]
proba = [(v[-1][fath][0] + log10(x1[sta.index(fath)][sta.index('E')]), len(string), fath) for fath in final_states]
exit_state = max(proba, key=itemgetter(0))
pointer = exit_state[1], exit_state[2]
while True:
    path.append(pointer[1])
    back = v[pointer[0]][pointer[1]]
    pointer = back[1], back[2]
    if pointer[1] == 'S0':
        break

print(' '.join(path[::-1]))

M1 D2 M3 M4


In [50]:
t=0.5 # the threshold 

obs=["A","T","G","C"] # observable states

true_mat=check(xx,t) # truth matrix based on no. og gaps in each column of alignments array

sta=states_define(len(xx[0])) # finds all hidden states in hmm profile array

x,y=profile_hmm(obs,xx,sta,true_mat)

x1,y1=add_pseudo(sta,len(xx[0]), x, y, 0.5)

string=["T","A","T","G","C"]# one insertion made

v=generate_viterbi(string,len(xx[0])-1, sta, x1, y1,obs)

path = list()
final_states = [tt+str(len(xx[0])) for tt in ['M', 'D', 'I']]
proba = [(v[-1][fath][0] + log10(x1[sta.index(fath)][sta.index('E')]), len(string), fath) for fath in final_states]
exit_state = max(proba, key=itemgetter(0))
pointer = exit_state[1], exit_state[2]
while True:
    path.append(pointer[1])
    back = v[pointer[0]][pointer[1]]
    pointer = back[1], back[2]
    if pointer[1] == 'S0':
        break

print(' '.join(path[::-1]))

M1 M2 M3 M4 I4


In [81]:
ali=np.array(["EEBBA--C-DBAA-AECD--BDB---CC-DDCBBCEDE-EBB-DAEE-C",
"EEEEABBCEABBCDEE-DAEBDBAEDC-BDBCB--C-B-BCA-DAEECA",
"--CEB-ACCDEACEEEEDBEED-ADBCCDAC--BC--BDBCAEDAEECC",
"A--AABDCE-A-CD-ECD-EBBA-EDC-DACCBBCCD-D-BA-DAAEBC",
"EECEAB--EDDACCE-CD-E--B-EDCCD-CCBBCCD-DBBA--AE-CA",
"E-CDA-DCECAAECB-CDCEB-B-BDCCD---B--CD-DBCDBDAEB-C",
"EBCEAEDC-DABC--A-DCEDDBAED-CD-CCBBCCEBDB--BEA-EEC",
"AC-E-BDCEDAADDEECDEEB-CAEDC-DD-CBBCCD-DBCABDAEECC",
"EECBABDCEDEAEC-DCDC-BDBDEDA-D-AD-A-EABEB--BDA-ECC"])

xx=[]
for i in ali:
    x=[]
    for j in range(len(ali[0])):
        x.append(i[j])
    xx.append(x)

t=0.359# the threshold 

obs=["A","B","C","D","E"] # observable states

true_mat=check(xx,t) # truth matrix based on no. og gaps in each column of alignments array

sta=states_define(len(xx[0])) # finds all hidden states in hmm profile array

x,y=profile_hmm(obs,xx,sta,true_mat)

x1,y1=add_pseudo(sta,len(xx[0]), x, y, 0.01)

string=[]
string1="EEBEABDCEEABCCCEEBDEDCADEDACCDCBBEECDBDACABDADCBE"# one insertion made
for i in string1:
    string.append(i)
    
v=generate_viterbi(string,len(xx[0])-1, sta, x1, y1,obs)

path = list()
final_states = [tt+str(len(xx[0])) for tt in ['M', 'D', 'I']]
proba = [(v[-1][fath][0] + log10(x1[sta.index(fath)][sta.index('E')]), len(string), fath) for fath in final_states]
exit_state = max(proba, key=itemgetter(0))
pointer = exit_state[1], exit_state[2]
while True:
    path.append(pointer[1])
    back = v[pointer[0]][pointer[1]]
    pointer = back[1], back[2]
    if pointer[1] == 'S0':
        break

print(' '.join(path[::-1]))

M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 M17 M18 M19 M20 M21 M22 M23 I23 M24 M25 M26 I26 I26 M27 M28 M29 M30 M31 M32 M33 M34 I34 M35 M36 M37 M38 I38 M39 M40 D41 M42 M43 M44 D45 D46 D47 D48 M49


In [75]:
len(xx[0])

49

In [76]:
len(string)

49

In [82]:
x1

array([[0.        , 0.01      , 0.89888889, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01      , 0.01      , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.01      ,
        0.01      ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01      ,
        0.01      ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])